# Notebook 2: Data Preparation

## Summary

This notebook cleans, transforms, and integrates all country-level datasets from Global Forest Watch. The main goal is to convert raw Excel data into a tidy, analysis-ready format.

## Key Activities

1. **Tree Cover Loss Processing**: 
   - Cleans column names and standardizes country names
   - Reshapes wide-format yearly columns (2001-2024) into long format
   - Output: `country_tree_cover_loss_processed.csv`

2. **Primary Forest Loss Processing**:
   - Fixes column naming inconsistencies (area__ha → area_ha)
   - Transforms yearly loss columns into tidy format
   - Output: `country_primary_loss_processed.csv`

3. **Drivers Processing**:
   - Pivots driver data from long to wide format (one column per driver)
   - Creates columns for each deforestation driver (logging, agriculture, wildfire, etc.)
   - Output: `country_drivers_processed.csv`

4. **Carbon Data Processing**:
   - Melts yearly carbon emission columns into long format
   - Extracts year information from column names
   - Output: `country_carbon_processed.csv`

5. **Data Integration**:
   - Merges all four processed datasets on country, threshold, and year
   - Replaces missing values with zeros
   - Output: `merged_clean_data.csv` (31,873 rows × 24 columns)

## Output Files

All processed files are saved to `data/processed/`:
- `country_tree_cover_loss_processed.csv`
- `country_primary_loss_processed.csv`
- `country_drivers_processed.csv`
- `country_carbon_processed.csv`
- `merged_clean_data.csv` (final integrated dataset)

---


### Loading Data and Printing sheets

In [1]:
import pandas as pd

RAW_PATH = "../data/raw/global_forest_watch_raw_data.xlsx"

# Read Excel workbook
global_forest_watch_excel_file = pd.ExcelFile(RAW_PATH)
global_forest_watch_excel_file.sheet_names


['Read_Me',
 'Country tree cover loss',
 'Country primary loss',
 'Country drivers',
 'Country carbon data',
 'Subnational 1 tree cover loss',
 'Subnational 1 primary loss',
 'Subnational 1 drivers',
 'Subnational 1 carbon data']

### 1. Cleaning the “Country Tree Cover Loss” Sheet

This dataset provides annual tree cover loss (in hectares) per country and canopy threshold between 2001–2024.
It represents total forest area lost, regardless of forest type or cause.

Our main goals for this section are to:
- Clean the data by standardizing column names, fixing country names, and converting data types.
- Reshape wide-format yearly columns (tc_loss_ha_2001, tc_loss_ha_2002, …) into a tidy long format with a single year column.
- Save the processed output to data/processed/country_tree_cover_loss_processed.csv.

In [18]:
import pandas as pd
import os
import re

RAW_PATH = "../data/raw/global_forest_watch_raw_data.xlsx"
global_forest_watch_excel_file = pd.ExcelFile(RAW_PATH)
country_tree_cover_loss_sheet = global_forest_watch_excel_file.parse("Country tree cover loss")

print("\n Preview of data (BEFORE processing):")
display(country_tree_cover_loss_sheet.head(10))

# Work on a copy to avoid touching raw data
tcl_clean = country_tree_cover_loss_sheet.copy()

# --- Standardize column names ---
tcl_clean.columns = tcl_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- Clean country names ---
if "country" in tcl_clean.columns:
    tcl_clean["country"] = tcl_clean["country"].astype(str).str.strip().str.title()

# --- Convert numeric columns properly ---
numeric_cols = tcl_clean.select_dtypes(include="object").columns
for col in numeric_cols:
    try:
        tcl_clean[col] = pd.to_numeric(tcl_clean[col])
    except (ValueError, TypeError):
        #keep it as-is
        pass
def melt_yearly_columns(df, prefix, value_name):
    """
    Converts wide year columns (e.g., tc_loss_ha_2001, tc_loss_ha_2002, …).
    """
    # Detect all columns that start with the prefix
    year_cols = [c for c in df.columns if c.startswith(prefix)]

    if not year_cols:
        print(f"No columns found with prefix '{prefix}'. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )
    # Extract year as integer from the column name
    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)

    # Drop the temporary column
    melted = melted.drop(columns=["metric_year"])

    return melted
# --- Apply transformation ---
tcl_tidy = melt_yearly_columns(tcl_clean, prefix="tc_loss_ha_", value_name="tree_cover_loss_ha")

print(" Transformed shape:", tcl_tidy.shape)
print("Columns:", list(tcl_tidy.columns)[:10])

print("\n Preview of tidy data (AFTER processing):")
display(tcl_tidy.head(10))

# Create folder if not already present
os.makedirs("../data/processed", exist_ok=True)

# Define output path
tcl_out_path = "../data/processed/country_tree_cover_loss_processed.csv"

# Save the processed tidy data
tcl_tidy.to_csv(tcl_out_path, index=False)

print(f"Saved processed dataset to: {tcl_out_path}")
print(f"Rows: {len(tcl_tidy):,} | Columns: {len(tcl_tidy.columns)}")

# Quick verification: reload and confirm structure
verify = pd.read_csv(tcl_out_path)
print("\nReloaded successfully! Sample below:")
display(verify.head(10))




 Preview of data (BEFORE processing):


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tc_loss_ha_2001,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Afghanistan,0,64383655,64383655,64383655,10738,103,214,267,226,...,0,0,0,31,25,46,47,16,133,223
1,Afghanistan,10,64383655,432070,126231,10738,92,190,254,207,...,0,0,0,28,19,40,37,9,32,32
2,Afghanistan,15,64383655,302629,106852,10738,91,186,248,205,...,0,0,0,28,19,39,32,7,23,17
3,Afghanistan,20,64383655,284330,105718,10738,89,181,245,203,...,0,0,0,28,18,39,32,7,22,16
4,Afghanistan,25,64383655,254843,72384,10738,89,180,244,202,...,0,0,0,27,18,38,27,6,21,14
5,Afghanistan,30,64383655,205771,71786,10738,88,179,244,201,...,0,0,0,26,18,36,26,6,15,10
6,Afghanistan,50,64383655,148417,46235,10738,78,135,200,159,...,0,0,0,20,15,33,23,5,8,6
7,Afghanistan,75,64383655,75480,18268,10738,46,61,96,61,...,0,0,0,9,8,20,9,2,3,3
8,Albania,0,2872761,2872761,2872761,16468,3907,940,675,3455,...,427,1394,2870,2926,1626,1829,3267,3516,3419,1631
9,Albania,10,2872761,838601,712542,16468,3815,909,636,3327,...,334,1108,2109,2345,1246,1310,2058,2441,2769,1093


 Transformed shape: (31872, 8)
Columns: ['country', 'threshold', 'area_ha', 'extent_2000_ha', 'extent_2010_ha', 'gain_2000-2012_ha', 'tree_cover_loss_ha', 'year']

 Preview of tidy data (AFTER processing):


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year
0,Afghanistan,0,64383655,64383655,64383655,10738,103,2001
1,Afghanistan,10,64383655,432070,126231,10738,92,2001
2,Afghanistan,15,64383655,302629,106852,10738,91,2001
3,Afghanistan,20,64383655,284330,105718,10738,89,2001
4,Afghanistan,25,64383655,254843,72384,10738,89,2001
5,Afghanistan,30,64383655,205771,71786,10738,88,2001
6,Afghanistan,50,64383655,148417,46235,10738,78,2001
7,Afghanistan,75,64383655,75480,18268,10738,46,2001
8,Albania,0,2872761,2872761,2872761,16468,3907,2001
9,Albania,10,2872761,838601,712542,16468,3815,2001


Saved processed dataset to: ../data/processed/country_tree_cover_loss_processed.csv
Rows: 31,872 | Columns: 8

Reloaded successfully! Sample below:


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year
0,Afghanistan,0,64383655,64383655,64383655,10738,103,2001
1,Afghanistan,10,64383655,432070,126231,10738,92,2001
2,Afghanistan,15,64383655,302629,106852,10738,91,2001
3,Afghanistan,20,64383655,284330,105718,10738,89,2001
4,Afghanistan,25,64383655,254843,72384,10738,89,2001
5,Afghanistan,30,64383655,205771,71786,10738,88,2001
6,Afghanistan,50,64383655,148417,46235,10738,78,2001
7,Afghanistan,75,64383655,75480,18268,10738,46,2001
8,Albania,0,2872761,2872761,2872761,16468,3907,2001
9,Albania,10,2872761,838601,712542,16468,3815,2001


### 2. Processing the “Country Primary Loss” Sheet

This dataset focuses on humid tropical primary forests, providing annual primary forest loss (in hectares) for 2002–2024.
It reflects the most ecologically significant areas of forest change.

Our main goals for this section are to:

- Audit the raw dataset to check column structure and detect anomalies.
- Clean column names (notably fixing area__ha → area_ha) and standardize country names.
- Reshape the yearly loss columns (tc_loss_ha_2002, tc_loss_ha_2003, …) into a tidy long format.

Save the processed output to data/processed/country_primary_loss_processed.csv.

In [9]:
# --- Load the sheet ---
country_primary_loss = global_forest_watch_excel_file.parse("Country primary loss")

print("\n Preview of data (BEFORE processing):")
display(country_primary_loss.head(10))

# Work on a copy
pl_clean = country_primary_loss.copy()

# --- Standardize column names ---
pl_clean.columns = pl_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- Rename inconsistent columns ---
if "area__ha" in pl_clean.columns:
    pl_clean = pl_clean.rename(columns={"area__ha": "area_ha"})
    print("Renamed column 'area__ha' → 'area_ha'")

# --- Clean country names ---
if "country" in pl_clean.columns:
    pl_clean["country"] = pl_clean["country"].astype(str).str.strip().str.title()

# --- Convert numeric-like columns ---
for col in pl_clean.columns:
    try:
        pl_clean[col] = pd.to_numeric(pl_clean[col])
    except (ValueError, TypeError):
        pass


def melt_yearly_columns(df, prefix, value_name):
    """
    Converts wide year columns (e.g., tc_loss_ha_2002, tc_loss_ha_2003, …)
    into a tidy long format with columns: country, threshold, year, <value_name>.
    """
    year_cols = [c for c in df.columns if c.startswith(prefix)]
    if not year_cols:
        print(f"No columns found with prefix '{prefix}'. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )

    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)
    melted = melted.drop(columns=["metric_year"])
    return melted

# --- Apply transformation ---
pl_tidy = melt_yearly_columns(pl_clean, prefix="tc_loss_ha_", value_name="primary_forest_loss_ha")

print("Transformed shape:", pl_tidy.shape)
print("Columns:", list(pl_tidy.columns)[:10])

print("\n📊 Preview of tidy data (after transformation):")
display(pl_tidy.head(10))

# Define output path
pl_out_path = "../data/processed/country_primary_loss_processed.csv"

# Save tidy dataset
pl_tidy.to_csv(pl_out_path, index=False)

print(f"💾 Saved processed dataset to: {pl_out_path}")
print(f"Rows: {len(pl_tidy):,} | Columns: {len(pl_tidy.columns)}")

# Verify save worked correctly
verify_pl = pd.read_csv(pl_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify_pl.head(5))





 Preview of data (BEFORE processing):


,country,threshold,area__ha,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,tc_loss_ha_2005,tc_loss_ha_2006,tc_loss_ha_2007,tc_loss_ha_2008,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Angola,30,2458061,3499,2963,2354,3110,1400,8060,2699,...,8998,12040,11166,13507,9995,8895,24326,15576,17627,13660
1,Argentina,30,4418724,9318,14459,28090,31429,24095,18687,47067,...,10547,15247,17202,9496,8983,20847,11921,21388,11473,12103
2,Australia,30,13977,0,0,0,0,25,0,0,...,5,0,0,0,5,0,0,0,0,0
3,Bangladesh,30,101114,619,266,347,306,677,369,240,...,205,345,414,358,387,459,308,307,743,467
4,Belize,30,1165487,5570,2993,2108,3206,1899,4140,3632,...,6606,11511,6616,4781,8772,16087,4560,4033,11667,21137
5,Benin,30,1952,0,0,0,0,0,0,0,...,2,1,1,0,2,0,0,0,0,0
6,Bhutan,30,1645545,119,84,337,315,256,186,319,...,278,703,790,406,294,472,399,513,658,449
7,Bolivia,30,40850721,70494,77076,96581,137883,118782,114368,180543,...,83291,245876,270242,154537,290650,276102,292264,384708,490542,1480900
8,Brazil,30,343260979,1621738,1570540,2016350,1824217,1415536,1149515,1075087,...,828839,2830943,2134474,1347176,1361053,1703491,1546964,1772214,1136250,2823646
9,Brunei,30,431532,474,379,584,602,442,639,760,...,806,1089,518,449,447,543,516,537,496,383


Renamed column 'area__ha' → 'area_ha'
Transformed shape: (1748, 5)
Columns: ['country', 'threshold', 'area_ha', 'primary_forest_loss_ha', 'year']

📊 Preview of tidy data (after transformation):


,country,threshold,area_ha,primary_forest_loss_ha,year
0,Angola,30,2458061,3499,2002
1,Argentina,30,4418724,9318,2002
2,Australia,30,13977,0,2002
3,Bangladesh,30,101114,619,2002
4,Belize,30,1165487,5570,2002
5,Benin,30,1952,0,2002
6,Bhutan,30,1645545,119,2002
7,Bolivia,30,40850721,70494,2002
8,Brazil,30,343260979,1621738,2002
9,Brunei,30,431532,474,2002


💾 Saved processed dataset to: ../data/processed/country_primary_loss_processed.csv
Rows: 1,748 | Columns: 5

✅ Reloaded successfully! Sample below:


,country,threshold,area_ha,primary_forest_loss_ha,year
0,Angola,30,2458061,3499,2002
1,Argentina,30,4418724,9318,2002
2,Australia,30,13977,0,2002
3,Bangladesh,30,101114,619,2002
4,Belize,30,1165487,5570,2002


In [8]:
# Work on a copy
pl_clean = pl_raw.copy()

# --- 1️⃣ Standardize column names ---
pl_clean.columns = pl_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Rename inconsistent columns ---
if "area__ha" in pl_clean.columns:
    pl_clean = pl_clean.rename(columns={"area__ha": "area_ha"})
    print("Renamed column 'area__ha' → 'area_ha'")

# --- 3️⃣ Clean country names ---
if "country" in pl_clean.columns:
    pl_clean["country"] = pl_clean["country"].astype(str).str.strip().str.title()

# --- 4️⃣ Replace empty strings with NaN ---
pl_clean = pl_clean.replace(r"^\s*$", pd.NA, regex=True)

# --- 5️⃣ Drop duplicates ---
before = len(pl_clean)
pl_clean = pl_clean.drop_duplicates(subset=["country", "threshold"], keep="first")
after = len(pl_clean)
print(f"Removed {before - after} duplicate rows (if any).")

# --- 6️⃣ Convert numeric-like columns ---
for col in pl_clean.columns:
    try:
        pl_clean[col] = pd.to_numeric(pl_clean[col])
    except (ValueError, TypeError):
        pass

# --- 7️⃣ Verify results ---
print("\n✅ After cleaning:")
display(pl_clean.info())
display(pl_clean.head(5))


Renamed column 'area__ha' → 'area_ha'
Removed 0 duplicate rows (if any).

✅ After cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          76 non-null     object
 1   threshold        76 non-null     int64 
 2   area_ha          76 non-null     int64 
 3   tc_loss_ha_2002  76 non-null     int64 
 4   tc_loss_ha_2003  76 non-null     int64 
 5   tc_loss_ha_2004  76 non-null     int64 
 6   tc_loss_ha_2005  76 non-null     int64 
 7   tc_loss_ha_2006  76 non-null     int64 
 8   tc_loss_ha_2007  76 non-null     int64 
 9   tc_loss_ha_2008  76 non-null     int64 
 10  tc_loss_ha_2009  76 non-null     int64 
 11  tc_loss_ha_2010  76 non-null     int64 
 12  tc_loss_ha_2011  76 non-null     int64 
 13  tc_loss_ha_2012  76 non-null     int64 
 14  tc_loss_ha_2013  76 non-null     int64 
 15  tc_loss_ha_2014  76 non-null     in

None

,country,threshold,area_ha,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,tc_loss_ha_2005,tc_loss_ha_2006,tc_loss_ha_2007,tc_loss_ha_2008,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Angola,30,2458061,3499,2963,2354,3110,1400,8060,2699,...,8998,12040,11166,13507,9995,8895,24326,15576,17627,13660
1,Argentina,30,4418724,9318,14459,28090,31429,24095,18687,47067,...,10547,15247,17202,9496,8983,20847,11921,21388,11473,12103
2,Australia,30,13977,0,0,0,0,25,0,0,...,5,0,0,0,5,0,0,0,0,0
3,Bangladesh,30,101114,619,266,347,306,677,369,240,...,205,345,414,358,387,459,308,307,743,467
4,Belize,30,1165487,5570,2993,2108,3206,1899,4140,3632,...,6606,11511,6616,4781,8772,16087,4560,4033,11667,21137


### 3. Processing the “Country Drivers” Sheet

This dataset links annual tree cover loss to its dominant drivers, such as agriculture, logging, fires, or urbanization.
Unlike the previous sheets, it already contains a driver and year column, so our main tasks are to:

- Audit the data for structure, missing values, and unique driver types.
- Clean column names and handle duplicates or blanks.
- Pivot the driver column into multiple columns (one per driver) to enable country-level comparisons.
- Save the cleaned and pivoted version into data/processed/country_drivers_processed.csv.

In [17]:
# --- Load raw 'Country drivers' sheet ---
country_drivers = global_forest_watch_excel_file.parse("Country drivers")

print("\n Preview of data (BEFORE processing):")
display(country_drivers.head(10))

# Work on a copy to keep raw data safe
drivers_clean = country_drivers.copy()

# --- Standardize column names ---
drivers_clean.columns = drivers_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- Clean country names ---
if "country" in drivers_clean.columns:
    drivers_clean["country"] = drivers_clean["country"].astype(str).str.strip().str.title()

# ---  Clean driver names ---
if "driver" in drivers_clean.columns:
    drivers_clean["driver"] = drivers_clean["driver"].astype(str).str.strip().str.title()

# --- Pivot drivers to wide format ---
drivers_pivot = (
    drivers_clean
    .pivot_table(
        index=["country", "threshold", "year"],
        columns="driver",
        values="tc_loss_ha",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
)

# --- Clean column names (make them lowercase, replace spaces/special chars with underscores) ---
drivers_pivot.columns = [
    re.sub(r"[^0-9A-Za-z_]+", "_", str(c)).lower().strip("_")
    for c in drivers_pivot.columns
]

print("\n Preview of pivoted data:")
display(drivers_pivot.head(10))


# Define output path
drivers_out_path = "../data/processed/country_drivers_processed.csv"

# Save the pivoted (wide) dataset
drivers_pivot.to_csv(drivers_out_path, index=False)

print(f" Saved processed dataset to: {drivers_out_path}")
print(f"Rows: {len(drivers_pivot):,} | Columns: {len(drivers_pivot.columns)}")

# Quick verification: reload to confirm structure
verify_drivers = pd.read_csv(drivers_out_path)
print("\n Reloaded successfully! Sample below:")
display(verify_drivers.head(5))



 Preview of data (BEFORE processing):


,country,threshold,driver,year,tc_loss_ha
0,Afghanistan,30,Hard commodities,2014,0.0
1,Afghanistan,30,Logging,2001,3.0
2,Afghanistan,30,Logging,2002,64.0
3,Afghanistan,30,Logging,2003,73.0
4,Afghanistan,30,Logging,2004,143.0
5,Afghanistan,30,Logging,2005,142.0
6,Afghanistan,30,Logging,2006,102.0
7,Afghanistan,30,Logging,2007,182.0
8,Afghanistan,30,Logging,2008,67.0
9,Afghanistan,30,Logging,2009,33.0



 Preview of pivoted data:


,country,threshold,year,hard_commodities,logging,other_natural_disturbances,permanent_agriculture,settlements_infrastructure,shifting_cultivation,wildfire
0,Afghanistan,30,2001,0.0,3.0,2.0,63.0,0.0,0.0,1.0
1,Afghanistan,30,2002,0.0,64.0,3.0,49.0,0.0,0.0,34.0
2,Afghanistan,30,2003,0.0,73.0,1.0,11.0,0.0,0.0,134.0
3,Afghanistan,30,2004,0.0,143.0,1.0,24.0,0.0,0.0,13.0
4,Afghanistan,30,2005,0.0,142.0,4.0,12.0,0.0,0.0,51.0
5,Afghanistan,30,2006,0.0,102.0,0.0,10.0,0.0,0.0,23.0
6,Afghanistan,30,2007,0.0,182.0,0.0,9.0,0.0,0.0,36.0
7,Afghanistan,30,2008,0.0,67.0,2.0,7.0,0.0,0.0,19.0
8,Afghanistan,30,2009,0.0,33.0,0.0,8.0,0.0,0.0,12.0
9,Afghanistan,30,2010,0.0,67.0,0.0,4.0,0.0,0.0,6.0


 Saved processed dataset to: ../data/processed/country_drivers_processed.csv
Rows: 3,625 | Columns: 10

 Reloaded successfully! Sample below:


,country,threshold,year,hard_commodities,logging,other_natural_disturbances,permanent_agriculture,settlements_infrastructure,shifting_cultivation,wildfire
0,Afghanistan,30,2001,0.0,3.0,2.0,63.0,0.0,0.0,1.0
1,Afghanistan,30,2002,0.0,64.0,3.0,49.0,0.0,0.0,34.0
2,Afghanistan,30,2003,0.0,73.0,1.0,11.0,0.0,0.0,134.0
3,Afghanistan,30,2004,0.0,143.0,1.0,24.0,0.0,0.0,13.0
4,Afghanistan,30,2005,0.0,142.0,4.0,12.0,0.0,0.0,51.0


###  4. Processing the “Country carbon data” Sheet
Why this step is important

This dataset contains information on forest-related carbon fluxes — including gross emissions, carbon removals, and net GHG balance per country and year.

The raw sheet stores each year as a separate column (e.g.,
gfw_forest_carbon_gross_emissions_2001__Mg_CO2e, ..._2002__Mg_CO2e, etc.),
we must first audit the structure, verify column patterns, and reshaping it into a tidy format.

In [20]:
# --- Load the raw sheet ---
carbon_raw = global_forest_watch_excel_file.parse("Country carbon data")

print("\n Preview of data (BEFORE processing):")
display(country_tree_cover_loss_sheet.head(10))

# Work on a copy to keep raw safe
carbon_clean = carbon_raw.copy()

# --- Standardize column names ---
carbon_clean.columns = carbon_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Rename threshold column (for consistency) ---
if "umd_tree_cover_density_2000__threshold" in carbon_clean.columns:
    carbon_clean = carbon_clean.rename(columns={"umd_tree_cover_density_2000__threshold": "threshold"})
    print("Renamed 'umd_tree_cover_density_2000__threshold' → 'threshold'")

# --- 3️⃣ Clean country names ---
if "country" in carbon_clean.columns:
    carbon_clean["country"] = carbon_clean["country"].astype(str).str.strip().str.title()


def melt_carbon_columns(df, pattern, value_name):
    """
    Melts all columns matching a yearly carbon emission pattern into a tidy long format.
    Example: gfw_forest_carbon_gross_emissions_2001__Mg_CO2e → year: 2001, value.
    """
    # Detect all year columns using regex pattern
    year_cols = [c for c in df.columns if re.match(pattern, c)]
    if not year_cols:
        print(" No matching year columns found. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )

    # Extract year from column names
    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)
    melted = melted.drop(columns=["metric_year"])

    return melted


# --- Apply transformation ---
carbon_tidy = melt_carbon_columns(
    carbon_clean,
    pattern=r"^gfw_forest_carbon_gross_emissions_\d{4}__mg_co2e$",
    value_name="carbon_gross_emissions_MgCO2e"
)

print(" Transformed shape:", carbon_tidy.shape)
print("Columns:", list(carbon_tidy.columns)[:10])

print("\nPreview of tidy data (after transformation):")
display(carbon_tidy.head(10))

# Define output path
carbon_out_path = "../data/processed/country_carbon_processed.csv"

# Save tidy dataset
carbon_tidy.to_csv(carbon_out_path, index=False)

print(f" Saved processed dataset to: {carbon_out_path}")
print(f"Rows: {len(carbon_tidy):,} | Columns: {len(carbon_tidy.columns)}")

# Verify save worked correctly
verify_carbon = pd.read_csv(carbon_out_path)
print("\n Reloaded successfully! Sample below:")
display(verify_carbon.head(5))





 Preview of data (BEFORE processing):


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tc_loss_ha_2001,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Afghanistan,0,64383655,64383655,64383655,10738,103,214,267,226,...,0,0,0,31,25,46,47,16,133,223
1,Afghanistan,10,64383655,432070,126231,10738,92,190,254,207,...,0,0,0,28,19,40,37,9,32,32
2,Afghanistan,15,64383655,302629,106852,10738,91,186,248,205,...,0,0,0,28,19,39,32,7,23,17
3,Afghanistan,20,64383655,284330,105718,10738,89,181,245,203,...,0,0,0,28,18,39,32,7,22,16
4,Afghanistan,25,64383655,254843,72384,10738,89,180,244,202,...,0,0,0,27,18,38,27,6,21,14
5,Afghanistan,30,64383655,205771,71786,10738,88,179,244,201,...,0,0,0,26,18,36,26,6,15,10
6,Afghanistan,50,64383655,148417,46235,10738,78,135,200,159,...,0,0,0,20,15,33,23,5,8,6
7,Afghanistan,75,64383655,75480,18268,10738,46,61,96,61,...,0,0,0,9,8,20,9,2,3,3
8,Albania,0,2872761,2872761,2872761,16468,3907,940,675,3455,...,427,1394,2870,2926,1626,1829,3267,3516,3419,1631
9,Albania,10,2872761,838601,712542,16468,3815,909,636,3327,...,334,1108,2109,2345,1246,1310,2058,2441,2769,1093


Renamed 'umd_tree_cover_density_2000__threshold' → 'threshold'
 Transformed shape: (11952, 10)
Columns: ['country', 'threshold', 'umd_tree_cover_extent_2000__ha', 'gfw_aboveground_carbon_stocks_2000__mg_c', 'avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1', 'gfw_forest_carbon_gross_emissions__mg_co2e_yr-1', 'gfw_forest_carbon_gross_removals__mg_co2_yr-1', 'gfw_forest_carbon_net_flux__mg_co2e_yr-1', 'carbon_gross_emissions_MgCO2e', 'year']

Preview of tidy data (after transformation):


,country,threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e,year
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,2001
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,2001
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,2001
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,2001
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,2001
5,Albania,75,363706,28761196,298,576299,3001723,-2425424,1137609.0,2001
6,Algeria,30,1223325,64822106,313,1872312,4873094,-3000781,574332.0,2001
7,Algeria,50,895366,50658903,334,1540229,3547408,-2007180,444098.0,2001
8,Algeria,75,496534,31035068,366,952542,1988182,-1035640,250927.0,2001
9,Angola,30,55276135,2879806419,296,62402574,170616018,-108213442,39294740.0,2001


 Saved processed dataset to: ../data/processed/country_carbon_processed.csv
Rows: 11,952 | Columns: 10

 Reloaded successfully! Sample below:


,country,threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e,year
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,2001
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,2001
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,2001
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,2001
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,2001


### 5. Integrating All Country-Level Datasets
Why this step is important

Up to this point, we have individually cleaned and reshaped four key country-level datasets from Global Forest Watch:

- Tree Cover Loss – annual loss of forested area
- Primary Forest Loss – loss in humid tropical primary forests
- Drivers of Deforestation – hectares of loss by cause (fire, agriculture, etc.)
- Carbon Data – annual forest-related CO₂ emissions and removals

Each dataset provides a different perspective on global forest change.
To perform meaningful Exploratory Data Analysis (EDA), predictive modeling, and visualization, we now need a single integrated dataset that combines all relevant variables per country and year.

This section merges the four processed datasets on their common identifiers —
country, threshold, and year — ensuring that all information aligns correctly in one master table.
Missing values will be preserved (NaN) so that no data is lost during integration.

In [24]:
# Load processed files
base_path = "../data/processed"

tcl = pd.read_csv(f"{base_path}/country_tree_cover_loss_processed.csv")
pl = pd.read_csv(f"{base_path}/country_primary_loss_processed.csv")
drv = pd.read_csv(f"{base_path}/country_drivers_processed.csv")
crb = pd.read_csv(f"{base_path}/country_carbon_processed.csv")

print(" Loaded all processed datasets:")
for name, df in zip(["Tree Cover Loss", "Primary Loss", "Drivers", "Carbon"], [tcl, pl, drv, crb]):
    print(f"{name:<15}: {df.shape}")

# --- Merge progressively on country + year (keeping all thresholds if present) ---
merged = (
    tcl
    .merge(pl, on=["country", "threshold", "year"], how="outer")
    .merge(drv, on=["country", "threshold", "year"], how="outer")
    .merge(crb, on=["country", "threshold", "year"], how="outer")
    .fillna(0)
)

print("\n Merged dataset shape:", merged.shape)
print("Columns:", merged.columns[:12].tolist(), "...")
display(merged.head(10))


# Define output path
merged_out_path = "../data/processed/merged_clean_data.csv"

# Save merged dataset
merged.to_csv(merged_out_path, index=False)

print(f"Final merged dataset saved to: {merged_out_path}")
print(f"Rows: {len(merged):,} | Columns: {len(merged.columns)}")

# Quick verification
verify_merged = pd.read_csv(merged_out_path)
print("\n Reloaded successfully! Sample below:")
display(verify_merged.head(10))


✅ Loaded all processed datasets:
Tree Cover Loss: (31872, 8)
Primary Loss   : (1748, 5)
Drivers        : (3625, 10)
Carbon         : (11952, 10)

✅ Merged dataset shape: (31873, 24)
Columns: ['country', 'threshold', 'area_ha_x', 'extent_2000_ha', 'extent_2010_ha', 'gain_2000-2012_ha', 'tree_cover_loss_ha', 'year', 'area_ha_y', 'primary_forest_loss_ha', 'hard_commodities', 'logging'] ...


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,settlements_infrastructure,shifting_cultivation,wildfire,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,172.0,2006,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,274.0,2007,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,123.0,2008,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,92.0,2009,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,109.0,2010,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


💾 Final merged dataset saved to: ../data/processed/merged_clean_data.csv
Rows: 31,873 | Columns: 24

✅ Reloaded successfully! Sample below:


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,settlements_infrastructure,shifting_cultivation,wildfire,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,172.0,2006,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,274.0,2007,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,123.0,2008,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,92.0,2009,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,109.0,2010,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
